# Основы подхода GitOps

## Базовая настройка Git

In [ ]:
git --version

In [ ]:
# export USER_NAME=''

In [ ]:
echo "${USER_NAME}"

In [ ]:
# export USER_EMAIL=''

In [ ]:
echo "${USER_EMAIL}"

Обязательно нужно указать кто будет автором изменений, которые совершаются на этом компьютере:

In [ ]:
git config --global user.name "${USER_NAME}"

А также email автора, чтобы с ним можно было связаться при желании:

In [ ]:
git config --global user.email "${USER_EMAIL}"

Полный список параметров git можно посмотреть вот так:

In [ ]:
git config --list

## Настройка GitLab

### Настройка ssh

In [ ]:
ssh-keygen -t ed25519 -f ~/.ssh/id_rsa -N '' -C "$(git config user.email)"

In [ ]:
ssh-keyscan gitlab >> ~/.ssh/known_hosts

### Аутентификация

1. Открыть GitLab: [http://localhost:8080](http://localhost:8080),
1. Username: `root`,
1. Password: `protected`.

### Добавление публичного ключа в GitLab

In [ ]:
cat ~/.ssh/id_rsa.pub

1. Открыть [GitLab](http://localhost:8080/-/profile/keys): `User` -> `Preferences` -> `SSH Keys` (Левая панель),
1. Нажать `Add new key`,
1. В поле `Key` ввести содержимое `~/.ssh/id_rsa.pub`,
1. Нажать `Add key`.

### Создание проекта

1. Открыть [GitLab](http://localhost:8080/projects/new): `Home Page` -> `Create a Project`,
1. Нажать [`Create blank project`](http://localhost:8080/projects/new#blank_project),
1. Значения:
    - `Project name`: `echo`,
    - `Pick a group or namespace (Project URL)`: выбрать `root`,
    - `Visibility Level`: `Public`,
    - **Снять** галочку `Initizalize repository with a README`.

## Локальная разработка

### Создание проекта

In [ ]:
mkdir ~/echo

In [ ]:
cd ~/echo

In [ ]:
cp -v ~/work/data/{Dockerfile,main.py,requirements.txt} .

In [ ]:
curl -k -Lo .gitignore 'https://www.toptal.com/developers/gitignore/api/jupyternotebooks,python,vim'

### Инициализация репозитория

In [ ]:
git init

In [ ]:
echo 'TBA' > README.md

In [ ]:
git add README.md

In [ ]:
git commit -m "Initial Commit"

In [ ]:
git add -A

In [ ]:
git status

In [ ]:
git commit -m "First placement of the project"

### Добавление внешнего репозитория

In [ ]:
git remote -v

In [ ]:
git remote add origin git@gitlab:root/echo.git

In [ ]:
git remote -v

In [ ]:
git fetch origin

In [ ]:
git status

### Публикация проекта

In [ ]:
git push origin master -u

### Добавление тестов

In [ ]:
git checkout -b add-tests

In [ ]:
echo 'assert True' > tests.py

In [ ]:
sed -i '/COPY main.py/s,COPY main.py,COPY main.py tests.py,' Dockerfile

In [ ]:
python tests.py

In [ ]:
echo 'assert False' > tests.py

In [ ]:
git status

In [ ]:
git diff

In [ ]:
git add -A

In [ ]:
git status

In [ ]:
git commit -m "Add tests"

In [ ]:
git log

In [ ]:
git push origin HEAD:add-tests

Создайте Merge Request и замержите его в `master`

### Обновить master после слияния

In [ ]:
git fetch origin

In [ ]:
git checkout master

In [ ]:
git merge origin/master

## Настройка GitLab CI

### Получить токен

1. Открыть [`Admin Area`](http://localhost:8080/admin) (Левая панель, первый комбо-бокс),
1. Нажать [`Shared Runners`](http://localhost:8080/admin/runners) (Средняя колонка, `Features` -> `Shared Runners`),
1. Нажать на кнопку с тремя вертикальными точками рядом с кнопкой `New instance runner`,
1. Нажать на кнопку `Copy registration token` рядом с кнопкой с иконкой с глазом,
1. Сохранить токен в переменную `RUNNER_TOKEN` ниже.

In [ ]:
RUNNER_TOKEN=FQBbsyXm5JmxBsmeDz92

### Конфигурация раннера

Раннер будет запускать docker контейнеры, которые должны быть в одной сети с GitLab, поэтому необходимо сохранить значение сети в переменную:

In [ ]:
GITLAB_NETWORK=$(docker inspect gitlab | sed -n '/git_network":/s,\s*"\([^"]\+\)".*,\1,p')

In [ ]:
gitlab-runner register \
  --non-interactive \
  --url http://gitlab/ \
  --registration-token ${RUNNER_TOKEN} \
  --name dockerfile-runner \
  --maintenance-note 'Dockerfile Runner' \
  --tag-list 'docker' \
  --docker-network-mode ${GITLAB_NETWORK} \
  --executor docker \
  --docker-privileged \
  --docker-volumes '/var/run/docker.sock:/var/run/docker.sock' \
  --docker-image docker:stable-dind

Проверьте, что раннер создался (обновить страницу с токеном): [`http://localhost:8080/admin/runners`](http://localhost:8080/admin/runners)

### Настройка проекта

In [ ]:
git checkout -b add-gitlab-ci master

In [ ]:
cp -v ~/work/data/.gitlab-ci.yml .

In [ ]:
cat .gitlab-ci.yml

In [ ]:
git status

In [ ]:
git add .gitlab-ci.yml

In [ ]:
git commit -m "Add .gitlab-ci.yml"

In [ ]:
git push origin HEAD:add-gitlab-ci

Откройте pipelines, чтобы увидеть, что CI запустился (левая панель, Build -> Pipelines): http://localhost:8080/root/echo/-/pipelines

**Внимание**: Пайплайн упадет, т.к. тесты в `tests.py` нерабочие.

Создайте Merge Request и замержите его, тесты будут исправлены в следующем шаге.

In [ ]:
git fetch origin
git checkout master
git merge origin/master

### Починка тестов

In [ ]:
git checkout -b fix-tests master

In [ ]:
echo 'assert True' > tests.py

In [ ]:
git add tests.py
git commit -m "Fix tests"

In [ ]:
git push origin HEAD:fix-tests

Проверьте пайплайн, тесты должны проходить успешно

Создайте Merge Request и замержите его в master.

In [ ]:
git fetch origin
git checkout master
git merge origin/master

## GitLab CD

После мержа в master запустился новый pipeline, который автоматически задеплоит приложение в виде докер контейнера: http://localhost:8000/?message=hello

Изменим логику:

In [ ]:
git checkout -b change-message master

In [ ]:
sed -i '/you_message/s,you_message,message,' main.py

In [ ]:
git diff

In [ ]:
git commit -am "Fix message" && git push origin HEAD:fix-message

Создайте Merge Request и замержите его в `master`. Откройте: http://localhost:8000/?message=hello

In [ ]:
git fetch origin
git checkout master
git merge origin/master